# DeepHeal Tutorial

This tutorial demonstrates how to use the `DeepHeal` package programmatically to learn low-dimensional representations of drug-induced proteomic changes.

In [ ]:
# Install DeepHeal if running in Google Colab
import sys
if "google.colab" in sys.modules:
    !pip install git+https://github.com/DeepHeal/DeepHeal.git

In [ ]:
import pandas as pd
import numpy as np
from deepheal.deepheal import DeepHeal
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

## 1. Create Synthetic Data

For this tutorial, we will create a synthetic dataset simulating proteomics log2 fold-changes. We will simulate 100 samples belonging to 3 different drug classes.

In [ ]:
# Parameters
n_samples = 100
n_features = 500
n_classes = 3

# Generate random features
X = np.random.randn(n_samples, n_features)

# Add some structure based on class
y = np.random.randint(0, n_classes, n_samples)
for i in range(n_samples):
    X[i, :] += y[i] * 0.5  # Shift mean based on class

# Create Sample IDs
sample_ids = [f"Sample_{i}" for i in range(n_samples)]
drug_classes = [f"Class_{label}" for label in y]

# Create Input DataFrame (Features)
input_df = pd.DataFrame(X, columns=[f"Prot_{i}" for i in range(n_features)])
input_df.insert(0, "Sample_ID", sample_ids)

# Create Meta DataFrame (Labels)
meta_df = pd.DataFrame({
    "Sample_ID": sample_ids,
    "Drug_Class": drug_classes
})

print("Input Data:")
print(input_df.head())
print("\nMeta Data:")
print(meta_df.head())

## 2. Initialize and Train DeepHeal

We initialize the model with desired hyperparameters. Here we use `no_batch=True` (implied by not providing a domain key or setting it to None) as we are doing simple dimensionality reduction.

In [ ]:
# Initialize model
model = DeepHeal(
    save_dir="tutorial_output",
    latent_dim=16,
    n_epochs=10,
    batch_size=32,
    lr=1e-3,
    verbose=True,
    use_importance_mask=True,
    domain_key=None, # No batch correction
)

# Prepare data (drop ID column from features)
features = input_df.drop(columns=["Sample_ID"])

# Set data
model.set_data(features, meta_data=meta_df)

# Train
model.train(save_model=True)

## 3. Generate Embeddings

After training, we can extract the low-dimensional embeddings for our samples.

In [ ]:
embeddings = model.predict(features)
print(f"Embeddings shape: {embeddings.shape}")

# Create a DataFrame for results
embedding_df = pd.DataFrame(embeddings, columns=[f"z{i+1}" for i in range(embeddings.shape[1])])
embedding_df["Drug_Class"] = meta_df["Drug_Class"]
embedding_df.head()

## 4. Visualization

Let's visualize the learned embeddings using PCA to see if the classes separate.

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(embeddings)

plt.figure(figsize=(8, 6))
for cls in np.unique(drug_classes):
    mask = np.array(drug_classes) == cls
    plt.scatter(pca_result[mask, 0], pca_result[mask, 1], label=cls, alpha=0.7)

plt.title("PCA of DeepHeal Embeddings")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

## 5. Feature Importance Analysis

Since we enabled `use_importance_mask=True`, we can analyze which features (proteins) were most important for the learned representation.

In [ ]:
import torch
from deepheal.utils import compute_feature_importance, prepare_ranked_list

# Ensure model is in eval mode and prepare input tensor
model.model.eval()
input_tensor = torch.tensor(features.values, dtype=torch.float32).to(model.config.device)

# Compute importance for the last layer of the encoder
# This gives us the contribution of input features to the latent representation
layer_idx = len(model.model.encoder) - 1
importance_matrix = compute_feature_importance(model.model, input_tensor, layer_idx)

print(f"Importance Matrix Shape: {importance_matrix.shape}")

# Prepare ranked lists of features
ranked_lists = prepare_ranked_list(importance_matrix, features.columns, expression_df=features)

# Display top 10 important features for the first neuron in the latent space
neuron_name = 'Neuron 0'
if neuron_name in ranked_lists:
    print(f"\nTop features for {neuron_name}:")
    print(ranked_lists[neuron_name].head(10))


## 6. Comparison with Other Methods

We compare DeepHeal with standard dimensionality reduction methods: PCA, NMF, and ICA.
We use the **Silhouette Score** to quantify how well the drug classes are separated in the low-dimensional space, and visualize the embeddings.

In [ ]:
from sklearn.decomposition import PCA, NMF, FastICA
from sklearn.metrics import silhouette_score

# Prepare Data
X = features.values
y_labels = meta_df["Drug_Class"]

# NMF requires non-negative data
X_pos = X - X.min()

# 1. Calculate Silhouette Scores (Quantitative)
# For fair comparison, we use the same latent dimension (16) for all methods
latent_dim = 16

# DeepHeal (already computed)
score_dh = silhouette_score(embeddings, y_labels)

# PCA
pca_16 = PCA(n_components=latent_dim).fit_transform(X)
score_pca = silhouette_score(pca_16, y_labels)

# NMF
nmf_16 = NMF(n_components=latent_dim, init='random', random_state=42, max_iter=500).fit_transform(X_pos)
score_nmf = silhouette_score(nmf_16, y_labels)

# ICA
ica_16 = FastICA(n_components=latent_dim, random_state=42, max_iter=500).fit_transform(X)
score_ica = silhouette_score(ica_16, y_labels)

print(f"Silhouette Score (DeepHeal): {score_dh:.4f}")
print(f"Silhouette Score (PCA):      {score_pca:.4f}")
print(f"Silhouette Score (NMF):      {score_nmf:.4f}")
print(f"Silhouette Score (ICA):      {score_ica:.4f}")

# 2. Visualization (2D)
# We project the embeddings to 2D for visualization.
# For DeepHeal, we use PCA on the 16D embeddings (consistent with Section 4).
# For baselines, we compute 2D projections directly from data.

pca_vis = PCA(n_components=2)
z_dh_2d = pca_vis.fit_transform(embeddings)

z_pca_2d = PCA(n_components=2).fit_transform(X)
z_nmf_2d = NMF(n_components=2, init='random', random_state=42, max_iter=500).fit_transform(X_pos)
z_ica_2d = FastICA(n_components=2, random_state=42, max_iter=500).fit_transform(X)

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
methods = [
    ("DeepHeal", z_dh_2d, score_dh),
    ("PCA", z_pca_2d, score_pca),
    ("NMF", z_nmf_2d, score_nmf),
    ("ICA", z_ica_2d, score_ica)
]

unique_classes = np.unique(y_labels)
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_classes)))

for ax, (name, z_2d, score) in zip(axes.flatten(), methods):
    for i, cls in enumerate(unique_classes):
        mask = (y_labels == cls)
        ax.scatter(z_2d[mask, 0], z_2d[mask, 1], label=cls, alpha=0.7, color=colors[i])
    ax.set_title(f"{name} (Silhouette: {score:.3f})")
    ax.set_xlabel("Dim 1")
    ax.set_ylabel("Dim 2")
    ax.legend()

plt.tight_layout()
plt.show()
